# Consultas en Elasticseacrh

In [18]:
from elasticsearch import Elasticsearch
from sqlalchemy import create_engine 
import re
from sentence_transformers import SentenceTransformer
import json

directory = "Guias Docentes"
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

es = Elasticsearch("http://elasticsearch:9200")
index = 'asignaturas_prueba'


## Buscar asignaturas que mencionen un tema en su descripción

In [20]:
import json

res = es.search(
    index=index,
    size=5,
    query={
        "match": {
            "descripcion_asignatura": "seguridad informática"
        }
    }
)

for hit in res.body["hits"]["hits"]:
    print(hit["_source"]["id_asignatura"], "-", hit["_source"]["descripcion_asignatura"], "\n")



615000236 - Proporcionar al estudiante una comprensión conceptual sólida de los fundamentos de la seguridad de la
información, incluyendo sus objetivos, servicios y amenazas principales, como primera toma de contacto con la
disciplina. Además, se busca introducir los principios de la criptografía mediante el estudio de técnicas clásicas
"útiles para entender los mecanismos básicos de cifrado" y presentar los fundamentos de la criptografía simétrica
moderna, con especial atención a sus algoritmos y modos de operación más representativos, sentando así las
bases para aprendizajes más avanzados en seguridad informática. 

615000224 - Proporcionar al estudiante una comprensión conceptual sólida de los fundamentos de la seguridad de la
información, incluyendo sus objetivos, servicios y amenazas principales, como primera toma de contacto con la
disciplina. Además, se busca introducir los principios de la criptografía mediante el estudio de técnicas clásicas
"útiles para entender los mecanismo

## Buscar por subtema dentro del temario (nested)

In [21]:
res = es.search(
    index=index,
    size=5,
    query={
        "nested": {
            "path": "temario.subtemas",
            "query": {
                "match": {
                    "temario.subtemas.titulo": "redes neuronales"
                }
            }
        }
    }
)

for hit in res.body["hits"]["hits"]:
    print(hit["_source"]["id_asignatura"])


615001054
615000246
615000309
615000715
615000349


## Buscar texto libre en varios campos

In [22]:
res = es.search(
    index=index,
    size=5,
    query={
        "multi_match": {
            "query": "machine learning avanzado",
            "fields": [
                "descripcion_asignatura",
                "competencias.texto",
                "conocimientos_previos"
            ]
        }
    }
)

for hit in res.body["hits"]["hits"]:
    print(hit["_source"]["id_asignatura"], "-", hit["_score"])


615001027 - 7.3907356
615000239 - 7.2631617
615000302 - 7.2631617
615000719 - 7.2631617
615000352 - 7.2631617


## Competencias que contengan cierta palabra

In [28]:
res = es.search(
    index=index,
    size=5,
    query={
        "nested": {
            "path": "competencias",
            "query": {
                "match": {
                    "competencias.texto": "trabajo en equipo"
                }
            },
            "inner_hits": {}
        }
    }
)

for hit in res.body["hits"]["hits"]:
    print(hit["_source"]["id_asignatura"])


615001034
615001039
615001017
615000721
615000307


## Buscar asignaturas con una competencia concreta por código

In [30]:
res = es.search(
    index=index,
    query={
        "nested": {
            "path": "competencias",
            "query": {
                "term": {
                    "competencias.codigo": "C"
                }
            }
        }
    }
)

for hit in res.body["hits"]["hits"]:
    print(hit["_source"]["id_asignatura"])

## Contar cuántas asignaturas hay por competencia

In [33]:
res = es.search(
    index=index,
    size=0,
    aggs={
        "competencias_mas_comunes": {
            "nested": {"path": "competencias"},
            "aggs": {
                "por_codigo": {
                    "terms": {"field": "competencias.codigo"}
                }
            }
        }
    }
)

for bucket in res.body["aggregations"]["competencias_mas_comunes"]["por_codigo"]["buckets"]:
    print(bucket["key"], "-", bucket["doc_count"])


CT2 - 32
CG01 - 27
CT8 - 22
CB02 - 21
CT4 - 21
CB01 - 20
CB05 - 20
CE1 - 20
CG06 - 20
CT6 - 20


## Obtener todas las asignaturas con sus temas principales

In [46]:
res = es.search(
    index=index,
    size=50,
    _source=["id_asignatura", "temario.titulo"],
    query={"match_all": {}}
)

for hit in res.body["hits"]["hits"]:
    print(hit["_source"]["id_asignatura"], "-", hit["_source"].get("temario", []), "\n")


615001026 - [{'titulo': 'Programación Matemática'}, {'titulo': 'Análisis en Redes'}, {'titulo': 'Programación Entera'}, {'titulo': 'Programación Multiobjetivo'}] 

615000364 - [{'titulo': 'Buenas prácticas en el diseño de bases de datos relacionales'}, {'titulo': 'Técnicas de optimización de consultas en BDR'}, {'titulo': 'Bases de datos distribuidas'}, {'titulo': 'Bases de Datos NoSql'}, {'titulo': 'Big Data y Business Intelligence'}, {'titulo': 'Cronograma'}] 

615000732 - [{'titulo': 'Tema 1: Introducción a los Sistemas de Comercio Electrónico.'}, {'titulo': 'Tema 2: Criptografía. PKI (Public Key Infrastructures). Certificados digitales (estándar X509).'}, {'titulo': 'Tema 3: Estándares de pago electrónico. Tarjetas inteligentes (Smart Cards).'}, {'titulo': "Tema 4: Implementación de tiendas virtuales -TV's-."}, {'titulo': 'Tema 5: Sistemas de autentificación biométricos.'}, {'titulo': 'Cronograma'}] 

615000204 - [{'titulo': '(INTRODUCCIÓN) CONJUNTOS, APLICACIONES Y RELACIONES'}, {

## Búsqueda por vectores

In [58]:
from sentence_transformers import SentenceTransformer
import json

model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

# Ejemplo de texto de búsqueda
query_text = "aprendizaje automático y redes neuronales"

# Generamos el vector
query_vector = model.encode(query_text).tolist()


In [74]:
query_text = "aprendizaje automático y redes neuronales"
query_vector = model.encode(query_text).tolist()

res = es.search(
    index=index,
    size=5,
    knn={
        "field": "descripcion_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 100
    },
    _source=["id_asignatura", "descripcion_asignatura"]
)

for hit in res.body["hits"]["hits"]:
    print(f"{hit['_source']['id_asignatura']} (score={hit['_score']:.4f})")
    print(hit["_source"].get("descripcion_asignatura", "")[:600], "\n")



615001033 (score=0.7566)
El aprendizaje automático es uno de los campos de estudio en inteligencia artificial que ha obtenido grandes
logros, por ejemplo, en problemas de visión artificial, de reconocimiento del lenguaje natural o de control de robots
autónomos. Dentro del aprendizaje automático, se pueden distinguir diferentes tipos de métodos que son
aplicables a diversas clases de problemas. Esta asignatura se centra principalmente en métodos basados en
redes de neuronas artificiales y en métodos de aprendizaje de modelos probabilísticos.
Dentro del enfoque basado en redes de neuronas artificiales, se presentan mét 

615001013 (score=0.6449)
La inteligencia artificial es una rama de la informática que estudia técnicas para simular el comportamiento
inteligente en un computador. En esta asignatura se estudian lenguajes formales de representación de diferentes
tipos de conocimiento así como sus modelos específicos de razonamiento para afrontar problemas reales.
Asimismo, se presentan 

In [77]:
query_text = "química"
query_vector = model.encode(query_text).tolist()

res = es.search(
    index=index,
    size=5,
    knn={
        "field": "descripcion_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 100
    },
    _source=["id_asignatura", "descripcion_asignatura"]
)

for hit in res.body["hits"]["hits"]:
    print(f"{hit['_source']['id_asignatura']} (score={hit['_score']:.4f})")
    print(hit["_source"].get("descripcion_asignatura", "")[:600], "\n")

615000232 (score=0.5717)
Los contenidos de esta asignatura pretenden ofrecer al alumno de primer curso las bases físicas y electrónicas de
la informática, estudiando los fenómenos eléctricos y magnéticos y los fundamentos de los circuitos eléctricos. 

615000212 (score=0.5717)
Los contenidos de esta asignatura pretenden ofrecer al alumno de primer curso las bases físicas y electrónicas de
la informática, estudiando los fenómenos eléctricos y magnéticos y los fundamentos de los circuitos eléctricos. 

615000220 (score=0.5717)
Los contenidos de esta asignatura pretenden ofrecer al alumno de primer curso las bases físicas y electrónicas de
la informática, estudiando los fenómenos eléctricos y magnéticos y los fundamentos de los circuitos eléctricos. 

615000702 (score=0.5717)
Los contenidos de esta asignatura pretenden ofrecer al alumno de primer curso las bases físicas y electrónicas de
la informática, estudiando los fenómenos eléctricos y magnéticos y los fundamentos de los circuitos el

In [83]:
query_text = "agricultura"
query_vector = model.encode(query_text).tolist()

res = es.search(
    index=index,
    size=5,
    knn={
        "field": "descripcion_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 100
    },
    _source=["id_asignatura", "descripcion_asignatura"]
)

for hit in res.body["hits"]["hits"]:
    print(f"{hit['_source']['id_asignatura']} (score={hit['_score']:.4f})")
    print(hit["_source"].get("descripcion_asignatura", "")[:600], "\n")

615000470 (score=0.5270)
El objeto de estudio teórico de la Inteligencia Artificial (IA) es la cognición, los procesos que permiten a un sistema
(biológico o no) percibir lo que ocurre en su entorno, razonar y actuar en ese mismo entorno. Este objeto de
estudio es extremadamente complejo y está enraizado en la cultura humana desde su nacimiento. Como no puede
ser de otro modo, en este estudio están interesadas muchas ramas de la ciencia y, por lo tanto, se trata de una
labor interdisciplinar en la que, además de la IA, se encuentran, al menos, la Psicología, la Filosofía, la Lingüística,
la Neurología, la Sociología y 

615000357 (score=0.5243)
La asignatura profundiza en los conceptos económico financieros que se iniciaron en la asignatura troncal de
Fundamentos de Economía y Empresa. Desarrolla en profundidad los conceptos de análisis estratégico en la
empresa y, en particular, aquellos pertenecientes al área de marketing.
Cursando esta asignatura el alumno puede obtener una panorámi

## Búsqueda semántica por competencias

In [85]:
query_text = "capacidad para resolver problemas matemáticos"
query_vector = model.encode(query_text).tolist()

res = es.search(
    index=index,
    size=5,
    knn={
        "field": "competencias_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 100
    },
    _source=["id_asignatura", "competencias.texto"]
)

for hit in res.body["hits"]["hits"]:
    print(f"{hit['_source']['id_asignatura']} (score={hit['_score']:.4f})")
    for comp in hit["_source"].get("competencias", []):
        print("-", comp["texto"])
    print()


615000229 (score=0.8009)
- Capacidad para la resolución de los problemas matemáticos que puedan plantarse en la ingeniería. Aptitud para aplicar los conocimientos sobre: algebra, cálculo diferencial e integral y métodos numéricos; estadística y optimización
- Resolución de problemas: Identificar, analizar y definir los elementos significativos que constituyen un problema para resolverlo con criterio y de forma efectiva

615000700 (score=0.7993)
- Capacidad para la resolución de los problemas matemáticos que puedan plantarse en la ingeniería. Aptitud para aplicar los conocimientos sobre: algebra, cálculo diferencial e integral y métodos numéricos; estadística y optimización.
- Resolución de problemas: Identificar, analizar y definir los elementos significativos que constituyen un problema para resolverlo con criterio y de forma efectiva.

615000207 (score=0.7993)
- Capacidad para la resolución de los problemas matemáticos que puedan plantarse en la ingeniería. Aptitud para aplicar los c

## Búsqueda semántica por conocimientos previos

In [87]:
query_text = "matemáticas básicas y álgebra"
query_vector = model.encode(query_text).tolist()

res = es.search(
    index=index,
    size=5,
    knn={
        "field": "conocimientos_previos_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 100
    },
    _source=["id_asignatura", "conocimientos_previos"]
)

for hit in res.body["hits"]["hits"]:
    print(f"{hit['_source']['id_asignatura']} (score={hit['_score']:.4f})")
    print(hit["_source"].get("conocimientos_previos", ""), "\n")


615000220 (score=0.7223)
- Analisis Matematico
3.2. Otros conocimientos previos recomendados para cursar la asignatura
- Conocimientos básicos de física: cinemática, dinámica, trabajo y energía.
- Haber cursado las asignaturas de física y matemáticas del bachillerato científico/tecnológico
- Conocimientos básicos de matemáticas: trigonometría, álgebra vectorial, cálculo diferencial e integral. 

615001052 (score=0.7098)
- Probabilidad Y Estadistica
- Analisis Matematico
- Fundamentos De Programacion
- Algebra
3.2. Otros conocimientos previos recomendados para cursar la asignatura
- We strongly recommend a good mathematical background (especially in linear algebra) and familiarity with
computational complexity and cryptography. 

615000239 (score=0.7097)
- Analisis Matematico
- Logica Y Matematica Discreta
3.2. Otros conocimientos previos recomendados para cursar la asignatura
- Cálculo elemental de probabilidades
- Cálculo diferencial.
- Series geométricas
- Función Gamma.
- Combinator

## Búsqueda semántica por descripcion_vector (con cosine_similarity)

In [ ]:
Como se usa cosine_similarity es necesario sumar 1 y multiplicar por 50 para pasar de la salida [-1,1] -> [0,2] -> [

In [98]:
query_text = "aprendizaje profundo y redes neuronales"
query_vector = model.encode(query_text, normalize_embeddings=True).tolist()

res = es.search(
    index=index,
    size=5,
    query={
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": """
                    double score = cosineSimilarity(params.query_vector, 'descripcion_vector');
                    return (score + 1) * 50;
                """,
                "params": {"query_vector": query_vector}
            }
        }
    },
    _source=["id_asignatura", "descripcion_asignatura"]
)

for hit in res.body["hits"]["hits"]:
    print(f"{hit['_source']['id_asignatura']} (score={hit['_score']:.4f})")
    print(hit["_source"].get("descripcion_asignatura", ""), "\n")



615001033 (score=65.4137)
El aprendizaje automático es uno de los campos de estudio en inteligencia artificial que ha obtenido grandes
logros, por ejemplo, en problemas de visión artificial, de reconocimiento del lenguaje natural o de control de robots
autónomos. Dentro del aprendizaje automático, se pueden distinguir diferentes tipos de métodos que son
aplicables a diversas clases de problemas. Esta asignatura se centra principalmente en métodos basados en
redes de neuronas artificiales y en métodos de aprendizaje de modelos probabilísticos.
Dentro del enfoque basado en redes de neuronas artificiales, se presentan métodos de aprendizaje profundo y
aprendizaje por refuerzo. Los algoritmos de aprendizaje profundo (deep learning en inglés) han mostrado su
capacidad para aprender de forma eficaz modelos a partir de conjuntos de datos de grandes dimensiones. En la
asignatura se presenta en primer lugar una visión general del aprendizaje profundo junto con estrategias para
entrenar redes ta

## Búsqueda semántica combinada por descripcion_vector, competencias_vector y conocimientos_previos_vector

In [109]:
query_text = "aprendizaje profundo y redes neuronales"
query_vector = model.encode(query_text, normalize_embeddings=True).tolist()

res = es.search(
    index=index,
    size=5,
    query={
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": """
                    double score = 0.5 * cosineSimilarity(params.query_vector, 'descripcion_vector') +
                                   0.3 * cosineSimilarity(params.query_vector, 'competencias_vector') +
                                   0.2 * cosineSimilarity(params.query_vector, 'conocimientos_previos_vector');
                    return (score + 1) * 50;  // desplazamos y escalamos
                """,
                "params": {"query_vector": query_vector}
            }
        }
    },
    _source=["id_asignatura", "descripcion_asignatura", "competencias.texto", "conocimientos_previos"]
)

for hit in res.body["hits"]["hits"]:
    print(f"{hit['_source']['id_asignatura']} (score={hit['_score']:.4f})")
    print("Descripción:", hit["_source"].get("descripcion_asignatura", "")[:600])
    print("Competencias:")
    for comp in hit["_source"].get("competencias", []):
        print("-", comp["texto"])
    print("Conocimientos previos:", hit["_source"].get("conocimientos_previos", ""), "\n")



615001033 (score=59.1391)
Descripción: El aprendizaje automático es uno de los campos de estudio en inteligencia artificial que ha obtenido grandes
logros, por ejemplo, en problemas de visión artificial, de reconocimiento del lenguaje natural o de control de robots
autónomos. Dentro del aprendizaje automático, se pueden distinguir diferentes tipos de métodos que son
aplicables a diversas clases de problemas. Esta asignatura se centra principalmente en métodos basados en
redes de neuronas artificiales y en métodos de aprendizaje de modelos probabilísticos.
Dentro del enfoque basado en redes de neuronas artificiales, se presentan mét
Competencias:
- Que los estudiantes hayan demostrado poseer y comprender conocimientos en un área de estudio que parte de la base de la educación secundaria general, y se suele encontrar a un nivel que, si bien se apoya en libros de texto avanzados, incluye también algunos aspectos que implican conocimientos procedentes de la vanguardia de su campo de estudi